In [1]:
import pandas as pd
import quandl , math
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import svm
import matplotlib.pyplot as plt
import pickle

In [2]:
file = "C:\\Users\\vinayak tyagi\\Downloads\\WIKI-PRICES.csv"

df = pd.read_csv(file)
df.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
0,A,1999-11-18,45.50,50.00,40.00,44.00,44739900.0,0.0,1.0,31.041951,34.112034,27.289627,30.018590,44739900.0
1,A,1999-11-19,42.94,43.00,39.81,40.38,10897100.0,0.0,1.0,29.295415,29.336350,27.160002,27.548879,10897100.0
2,A,1999-11-22,41.31,44.00,40.06,44.00,4705200.0,0.0,1.0,28.183363,30.018590,27.330562,30.018590,4705200.0
3,A,1999-11-23,42.50,43.63,40.25,40.25,4274400.0,0.0,1.0,28.995229,29.766161,27.460188,27.460188,4274400.0
4,A,1999-11-24,40.13,41.94,40.00,41.06,3464400.0,0.0,1.0,27.378319,28.613174,27.289627,28.012803,3464400.0


In [3]:
df = df[['adj_open','adj_high','adj_low','adj_close','adj_volume']]
print(df.head())

    adj_open   adj_high    adj_low  adj_close  adj_volume
0  31.041951  34.112034  27.289627  30.018590  44739900.0
1  29.295415  29.336350  27.160002  27.548879  10897100.0
2  28.183363  30.018590  27.330562  30.018590   4705200.0
3  28.995229  29.766161  27.460188  27.460188   4274400.0
4  27.378319  28.613174  27.289627  28.012803   3464400.0


In [4]:
df["HL_PCT"] = (df['adj_high'] - df['adj_close'])/df['adj_close'] * 100
df["PCT_change"] = (df['adj_close'] - df['adj_open'])/df['adj_open'] * 100

df = df[["adj_close","HL_PCT","PCT_change","adj_volume"]]
print(df.head())

   adj_close     HL_PCT  PCT_change  adj_volume
0  30.018590  13.636364   -3.296703  44739900.0
1  27.548879   6.488361   -5.961807  10897100.0
2  30.018590   0.000000    6.511740   4705200.0
3  27.460188   8.397516   -5.294118   4274400.0
4  28.012803   2.143205    2.317468   3464400.0


In [5]:
forecast_col = 'adj_close'
df.fillna(-99999, inplace = True)

forecast = int(math.ceil(0.01*len(df)))

df['label'] = df[forecast_col].shift(-forecast)
df.dropna(inplace = True)

print(df.head())

   adj_close     HL_PCT  PCT_change  adj_volume      label
0  30.018590  13.636364   -3.296703  44739900.0  58.672699
1  27.548879   6.488361   -5.961807  10897100.0  61.401662
2  30.018590   0.000000    6.511740   4705200.0  56.414482
3  27.460188   8.397516   -5.294118   4274400.0  54.238135
4  28.012803   2.143205    2.317468   3464400.0  59.354940


In [6]:
X = np.array(df.drop(['label'],1))
y = np.array(df['label'])

X = preprocessing.scale(X)
print(X,y)
print(len(X),len(y))

[[-0.24335685  3.95431101 -0.9010575   8.7261177 ]
 [-0.26100841  1.50847719 -1.61278172  1.49730771]
 [-0.24335685 -0.71164662  1.71831767  0.17472009]
 ...
 [-0.43920207 -0.71164662  0.33540806 -0.81531426]
 [-0.43920207 -0.71164662 -0.02066272 -0.83003069]
 [-0.43969428  0.21313778 -0.72343399 -0.75851752]] [58.672699   61.40166175 56.41448233 ...  1.99716609  2.03159999
  2.10046779]
9900 9900


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2) 

clf = LinearRegression()
clf.fit(X_train,y_train)

# THIS IS USED SAVED THE MODEL NO NEED TO TRAIN AGAIN 
with open('stockPrediction_lr.pickle','wb') as f:
    pickle.dump(clf,f)

pickle_in = open("stockPrediction_lr.pickle","rb")
clf = pickle.load(pickle_in)

acc = clf.score(X_test,y_test)
print(acc)

0.6590938839767342


In [8]:
# predicting the vlaue
X_Z = X[-forecast:]
z = clf.predict(X_Z)
print(z)

[19.07257388 15.86484644 15.98579774 18.12689662 16.00654385 17.32318692
 15.48953604 17.30580548 15.71233938 17.27648426 18.18091565 17.28533728
 17.24545736 17.54990901 15.62923647 17.27395559 17.241223   17.27648426
 15.04928297 15.96785822 19.09302297 17.19752444 17.17023351 14.00570935
 17.25057441 16.47535002 17.27395559 17.24545736 17.24975141 16.62456746
 18.1040127  17.24886872 17.25057441 18.18862108 17.29474839 17.30580548
 17.27477858 17.27818995 16.65415324 15.07998527 17.32318692 17.34568541
 16.12110984 20.0418028  19.1508111  15.52240491 17.46979304 15.43606329
 16.38886432 17.43062772 15.92822609 17.38441774 15.25249058 17.29011248
 17.23790276 15.05996755 14.95443133 15.52485033 19.57275304 17.30157113
 17.30580548 15.68334276 17.27566127 16.03352791 14.44033286 17.25057441
 16.65979051 17.28874865 17.21222638 15.78364442 17.24545736 19.06743772
 13.1229314  15.74008262 18.19182554 17.26113311 15.69658111 15.97126958
 17.27566127 17.2119929  14.15922085 17.16870161 15